In [19]:
# -- LIBRAIRIES & INCLUDE FUNCTION FILES --#

import XLSX
import DataFrames
include("building_profiles.jl")

build_daily_LCT_profiles (generic function with 1 method)

In [13]:
# -- LOAD PROFILES PATH --#

PROF_PATH = joinpath(splitdir(@__DIR__)[1], "Manchester_data/LCT_profiles")


load_files = [joinpath(PROF_PATH, "Summer_Load_Profiles.xlsx"),
              joinpath(PROF_PATH, "Winter_Load_Profiles.xlsx")
              ]

LCT_files = [joinpath(PROF_PATH, "Summer_PV_Profiles.xlsx"),
             joinpath(PROF_PATH, "Summer_EHP_Profiles.xlsx"),
             joinpath(PROF_PATH, "Winter_EV_Profiles.xlsx"),
             joinpath(PROF_PATH, "Winter_uCHP_Profiles.xlsx")
             ]

4-element Vector{String}:
 "/Users/manoncornet/Documents/Un" ⋯ 39 bytes ⋯ "rofiles/Summer_PV_Profiles.xlsx"
 "/Users/manoncornet/Documents/Un" ⋯ 40 bytes ⋯ "ofiles/Summer_EHP_Profiles.xlsx"
 "/Users/manoncornet/Documents/Un" ⋯ 39 bytes ⋯ "rofiles/Winter_EV_Profiles.xlsx"
 "/Users/manoncornet/Documents/Un" ⋯ 41 bytes ⋯ "files/Winter_uCHP_Profiles.xlsx"

In [14]:
# -- NETWORK FILE PATH --

XLSX_FILE_PATH = joinpath(splitdir(@__DIR__)[1], "network_models/network_Nahman_Peric_2S23H.xlsx")

"/Users/manoncornet/Documents/University/TFE/DNEP/network_models/network_Nahman_Peric_2S23H.xlsx"

In [38]:
# -- SCALING THE LOAD PROFILE DATA --

df_bus  = DataFrames.DataFrame(XLSX.readtable(XLSX_FILE_PATH, "bus"))
Ns_size = sum(df_bus.type .== "substation") # nb of substation buses
Nu_size = sum(df_bus.type .== "user")       # nb of load nodes
Nu = (1:Nu_size) .+ Ns_size                 # set of load buses     

# Get the max power demand repartition among the nodes in MVA
cos_phi = 1
S_D     = convert(Vector{Float64}, df_bus.S_D_mva)[Nu]  # [MVA]
P_D     = S_D * cos_phi                                 # [MW]
Q_D     = S_D * sin(acos(cos_phi))                      # [MVar]                            

# The Manchester load profiles are in kW => active power 
load_profiles = build_daily_load_profiles(load_files; nb_loads = 21) .* 1e-3 # [MW]

function norm_col(array::Array{Float64})
    return array ./ maximum(array, dims=1)
end

norm_load_profiles = norm_col(load_profiles)

scaled_load_profiles = norm_load_profiles .* P_D'
println(maximum(scaled_load_profiles, dims=1))

[0.64 0.32 0.32 0.32 0.32 0.32 0.32 0.32 0.32 0.32 0.32 0.32 0.32 0.32 0.32 0.32 0.32 0.32 0.32 0.32 0.32]
